In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib.patches as patches
from rtseg.utils.param_io import load_params
from scipy.signal import find_peaks, peak_prominences
%matplotlib qt5

### Load image

In [3]:
phase_img_path = Path('/mnt/sda1/REALTIME/data/test_data/EXP-24-CD0103/40x newchip testimages/_1/Default/phase/img_channel000_position000_time000000000_z000.tif')

In [4]:
image = imread(phase_img_path)

### Get barcodes first and then figure out the channel locations to cut

In [5]:
from rtseg.identify_channels import get_barcode_model, detect_barcodes

In [6]:
barcode_model_path = Path("/mnt/sda1/REALTIME/network_expts/barcode_expts/2025-01-17_13-26-07/model.pth")
params_path = Path("/home/pk/Documents/rtseg/rtseg/resources/reference_params/reference_linux.yaml")
params = load_params(params_path, ref_type="expt")
barcode_params = params.BarcodeAndChannels
anchor_sizes = barcode_params.model_params.anchors.sizes
strides = barcode_params.model_params.anchors.strides
num_classes = barcode_params.model_params.num_classes
device = 'cuda:0'

In [7]:
model, anchors, strides = get_barcode_model(barcode_model_path, anchor_sizes, strides)

In [8]:
bboxes_final = detect_barcodes(model, anchors, strides, image.astype('float32'), [256, 800])

In [9]:
fig, ax = plt.subplots()
ax.imshow(image, cmap='gray')
for row in bboxes_final:
    rect = patches.Rectangle((row[0], row[1]), row[2] - row[0], row[3] - row[1], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
plt.show()

### Cutting channels

In [10]:
bboxes_final

[array([611.7605   , 212.61502  , 652.5735   , 558.02313  ,   0.8009013,
          0.       ], dtype=float32),
 array([1.2066503e+03, 2.2793683e+02, 1.2445505e+03, 5.7554742e+02,
        5.3137660e-01, 0.0000000e+00], dtype=float32),
 array([1.7980172e+03, 2.5303416e+02, 1.8349648e+03, 5.9444427e+02,
        8.5779959e-01, 0.0000000e+00], dtype=float32),
 array([2.3908232e+03, 2.8842435e+02, 2.4292185e+03, 6.1529254e+02,
        7.2835815e-01, 0.0000000e+00], dtype=float32),
 array([2.9868606e+03, 2.8449405e+02, 3.0191638e+03, 6.2564075e+02,
        3.9682052e-01, 0.0000000e+00], dtype=float32)]

In [11]:
bbox_centers = [(bbox[0] + bbox[2])/2 for bbox in bboxes_final]

In [12]:
bbox_centers

[632.1669921875,
 1225.600341796875,
 1816.490966796875,
 2410.02099609375,
 3003.01220703125]

#### Strategy

1. Find peaks
2. Find the peaks that lie inside the barcode bboxes
3. Go to 14 peaks on the left of each barcode bbox
4. Check that the first bbox is more than a block from the start of the image.
5. finalize channel locations

In [13]:
hist = np.sum(image, axis=0)

In [14]:
plt.figure()
plt.plot(hist)
plt.show()

In [15]:
peaks, _ = find_peaks(hist, distance=23)

In [16]:
peaks

array([   7,   35,   65,  108,  149,  190,  232,  274,  315,  357,  398,
        438,  481,  522,  564,  605,  633,  660,  703,  744,  785,  825,
        867,  907,  948,  990, 1032, 1072, 1114, 1155, 1196, 1224, 1251,
       1293, 1334, 1375, 1415, 1457, 1499, 1540, 1582, 1622, 1663, 1704,
       1744, 1786, 1817, 1842, 1884, 1924, 1966, 2007, 2048, 2089, 2130,
       2173, 2214, 2255, 2297, 2338, 2380, 2406, 2436, 2478, 2519, 2559,
       2601, 2642, 2685, 2726, 2768, 2808, 2850, 2891, 2933, 2975, 3002])

In [17]:
len(peaks)

77

In [18]:
plt.figure()
plt.plot(hist)
plt.plot(peaks, hist[peaks],'r*')
plt.show()

#### Find peaks the lie inside barcodes

In [19]:
bboxes_final

[array([611.7605   , 212.61502  , 652.5735   , 558.02313  ,   0.8009013,
          0.       ], dtype=float32),
 array([1.2066503e+03, 2.2793683e+02, 1.2445505e+03, 5.7554742e+02,
        5.3137660e-01, 0.0000000e+00], dtype=float32),
 array([1.7980172e+03, 2.5303416e+02, 1.8349648e+03, 5.9444427e+02,
        8.5779959e-01, 0.0000000e+00], dtype=float32),
 array([2.3908232e+03, 2.8842435e+02, 2.4292185e+03, 6.1529254e+02,
        7.2835815e-01, 0.0000000e+00], dtype=float32),
 array([2.9868606e+03, 2.8449405e+02, 3.0191638e+03, 6.2564075e+02,
        3.9682052e-01, 0.0000000e+00], dtype=float32)]

In [20]:
bboxes_taken = [False for bbox in bboxes_final]
bboxes_bounds = [(bbox[0], bbox[2]) for bbox in bboxes_final]

In [21]:
bboxes_bounds

[(611.7605, 652.5735),
 (1206.6503, 1244.5505),
 (1798.0172, 1834.9648),
 (2390.8232, 2429.2185),
 (2986.8606, 3019.1638)]

In [22]:
bboxes_taken

[False, False, False, False, False]

In [23]:
channel_locations = dict()
for peak_idx, peak in enumerate(peaks, 0):
    for i, bbox_bound in enumerate(bboxes_bounds):
        if peak >= bbox_bound[0] and peak <= bbox_bound[1]:
            print(peak, peak_idx,"--->", bbox_bound, i)
            try:
                channel_locations[i] = peaks[peak_idx-14: peak_idx]
            except:
                channel_locations[i] = np.array([])

633 16 ---> (611.7605, 652.5735) 0
1224 31 ---> (1206.6503, 1244.5505) 1
1817 46 ---> (1798.0172, 1834.9648) 2
2406 61 ---> (2390.8232, 2429.2185) 3
3002 76 ---> (2986.8606, 3019.1638) 4


In [24]:
channel_locations

{0: array([ 65, 108, 149, 190, 232, 274, 315, 357, 398, 438, 481, 522, 564,
        605]),
 1: array([ 660,  703,  744,  785,  825,  867,  907,  948,  990, 1032, 1072,
        1114, 1155, 1196]),
 2: array([1251, 1293, 1334, 1375, 1415, 1457, 1499, 1540, 1582, 1622, 1663,
        1704, 1744, 1786]),
 3: array([1842, 1884, 1924, 1966, 2007, 2048, 2089, 2130, 2173, 2214, 2255,
        2297, 2338, 2380]),
 4: array([2436, 2478, 2519, 2559, 2601, 2642, 2685, 2726, 2768, 2808, 2850,
        2891, 2933, 2975])}

#### Putting together

In [ ]:
from rtseg.identify_channels import channel_locations

In [26]:
channel_locations(image, bboxes_final)

633 16 ---> (611.7605, 652.5735) 0
1224 31 ---> (1206.6503, 1244.5505) 1
1817 46 ---> (1798.0172, 1834.9648) 2
2406 61 ---> (2390.8232, 2429.2185) 3
3002 76 ---> (2986.8606, 3019.1638) 4


({0: array([], dtype=float64),
  1: array([], dtype=float64),
  2: array([], dtype=float64),
  3: array([], dtype=float64),
  4: array([], dtype=float64)},
 [True, True, True, True, True])